In [1]:
!pip install keras_cv_attention_models -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.4.0 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
kfp 1.8.20 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.86.0 which is incompatible.
kfp 1.8.20 requires PyYAML<6,>=5.3, but you have pyyaml 6.0 which is incompatible.
gcsfs 2023.3.0 requires fsspec==2023.3.0, but you have fsspec 2023.4.0 which is incompatible.
cudf 23.4.0 requires protobuf<4.22,>=4.21.6, but you have protobuf 3.19.6 which is incompatible.
beatrix-jupyterlab 2023.46.184821 requires jupyter-server~=1.16, but you have jupyter-server 2.5.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_cv_attention_models import convnext

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/train/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/test/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [5]:
convnext_base = convnext.ConvNeXtLarge(input_shape=(128, 128, 3), pretrained=None, num_classes=0)
x = tf.keras.layers.GlobalAveragePooling2D()(convnext_base.output)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
convnext_base_model = tf.keras.Model(convnext_base.input, x)

In [6]:
METRICS = [
        'accuracy',
    ]
    
convnext_base_model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=METRICS
    )

In [7]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [8]:
history = convnext_base_model.fit(
    train_generator,
    epochs=8,
    validation_data=validation_generator,
    callbacks=[lr_scheduler]
)

Epoch 1/8


2023-05-03 09:13:07.271697: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/stack1_block2_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


652/652 [==============================] - 244s 261ms/step - loss: 22.9183 - accuracy: 0.6979 - val_loss: 48.2904 - val_accuracy: 0.3750 - lr: 0.0100
Epoch 2/8
652/652 [==============================] - 160s 246ms/step - loss: 68.1244 - accuracy: 0.6204 - val_loss: 2.7274 - val_accuracy: 0.6250 - lr: 0.0089
Epoch 3/8
652/652 [==============================] - 161s 247ms/step - loss: 10.6543 - accuracy: 0.6267 - val_loss: 1.5338 - val_accuracy: 0.6250 - lr: 0.0079
Epoch 4/8
652/652 [==============================] - 161s 247ms/step - loss: 2.9135 - accuracy: 0.6308 - val_loss: 0.6983 - val_accuracy: 0.6250 - lr: 0.0071
Epoch 5/8
652/652 [==============================] - 162s 249ms/step - loss: 5.1191 - accuracy: 0.6323 - val_loss: 1.8912 - val_accuracy: 0.6250 - lr: 0.0063
Epoch 6/8
652/652 [==============================] - 162s 248ms/step - loss: 1.0686 - accuracy: 0.6384 - val_loss: 0.6794 - val_accuracy: 0.6250 - lr: 0.0056
Epoch 7/8
652/652 [==============================] - 163s 

In [9]:
convnext_base_model.save("CovnNeXT_xray_Pneumonia_large.h5")